In [16]:
# doccano.json einlesen
import json
import spacy
from spacy.tokens import DocBin

# --- Options: ---
# sortiert Label anhand der shouldBe-Listen wenn nötig um
rearrange = True
# exportiert die entstandenen Daten als jsonl
exportData = True
# printet alle neu konvertierten Labelpositionen  
printLabels = False
# konvertiert File in besser lesbar. Dann aber nicht mehr in spaCy-Format umwandelbar
readable = False
# maximale Anzahl von Wörtern um die die Spangrenze (Ende) nach rechts verschoben werden kann
# Wird benötigt, um Fehler zu vermeiden
maxCounter = 2

fromModelNumber = "3"
ToModelNumber = "4"
datasetToTransform = "editedData" #"dev"
    

In [17]:
#erstelle Doc mit dem text und füge alle Annotations hinzu 
with open('./corpus/Modell_{}/{}.jsonl'.format(fromModelNumber, datasetToTransform), 'r') as f:
        for cv in f:
            currentCV = json.loads(cv)
            cvList = currentCV["CVs"]

# leeres neues Modell erstellen
nlp = spacy.blank("de")

lineDicts = []
dicts = {"lines": lineDicts}

lastAnnotation = None

for cv in cvList:
    text = cv['text']
    annotations = cv['label']
    annotations.sort()
    doc = nlp(text)
    lines = text.split("\n") 
    
    start = 0
    end = -1
    
    for line in lines:
        start = end + 1
        end = start + len(line)
        labels = []
        
        for annotation in annotations:
            labelStart = int(annotation[0])
            labelEnd = int(annotation[1])
            label = annotation[2]
            
            #löscht die letzte hinzugefügte Annotation aus annotations
            if lastAnnotation != None:
                annotations.remove(lastAnnotation)
                lastAnnotation = None
                
            if labelStart >= start and labelEnd <= end+1:
                #print("Line len: ", len(line), ";Start: ", start, "; End: ", end)
                span = doc.char_span(labelStart, labelEnd, label=label)
                if printLabels:
                    print("Line: -{}-".format(line))
                    print("Labled Text: ", span)
                    print("Vorher: labelStart: ", labelStart, "; labelEnd: ", labelEnd)
                labelStart = labelStart - start
                labelEnd = labelEnd - start
                labels.append([labelStart, labelEnd, label])
                lastAnnotation = annotation
                if printLabels:
                    print("Vorher: labelStart: ", labelStart, "; labelEnd: ", labelEnd)
                    print("end: ", end)
                    print("Remove Anno: ", lastAnnotation)
                    print("---------------\n")
            # damit nicht unnötig jedes mal alle Label durchgegangen werden
            elif(labelStart > end):
                if len(labels) > 0 and printLabels:
                    print("Line: -{}-".format(line))
                    print("Breaks at: ", annotation)
                    print("   end: ", end)
                    print("---------------\n")
                break
                
        line = line + "\n"
        lineDicts.append({"text": line, "label": labels})

if exportData:
    fileName = "./corpus/Modell_{}/{}.jsonl".format(ToModelNumber, datasetToTransform)
    with open(fileName, 'w', encoding='utf-8') as f:
        if readable:
            json.dump(dicts, f, ensure_ascii=False, indent=2)
        else:
            json.dump(dicts, f, ensure_ascii=False)
